In [1]:
# tests/test_routes_provider_manual.py
# Run:  python -m tests.test_routes_provider_manual

import os, sys
import json
import time
import traceback

# Optional: load .env for local runs
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

# Choose the import that matches your project layout:
os.chdir("..")  # jump from tests/ to project root
sys.path.insert(0, os.getcwd())
from modules.routes_provider import ORSConfig, ORSClient
# from cabosupernet.routes_provider import ORSConfig, ORSClient

def jprint(obj) -> None:
    """Pretty-print dicts/lists like JSON."""
    print(json.dumps(obj, ensure_ascii=False, indent=2, sort_keys=True))

def run_case(label: str, origin, destination, **kwargs):
    """Run one routing case with timing and robust error display."""
    print("\n" + "=" * 80)
    print(f"{label}")
    print("-" * 80)
    print("origin:", origin)
    print("destination:", destination)
    t0 = time.time()
    try:
        resp = ors.route_road(origin, destination, **kwargs)
        elapsed = (time.time() - t0) * 1000
        print(f"\n✓ OK in {elapsed:.0f} ms — result:\n")
        jprint(resp)
    except Exception as e:
        elapsed = (time.time() - t0) * 1000
        print(f"\n✗ ERROR in {elapsed:.0f} ms — details:\n")
        # If it's an HTTP error, try to show server JSON/text body
        body = None
        resp = getattr(e, "response", None)
        if resp is not None:
            try:
                body = resp.json()
            except Exception:
                body = resp.text
        err_info = {
            "error_type": type(e).__name__,
            "http_status": getattr(resp, "status_code", None),
            "server_body": body,
            "exception_str": str(e),
        }
        jprint(err_info)
        print("\nTraceback (for debugging):")
        traceback.print_exc()

# Guard: require ORS_API_KEY
if not os.getenv("ORS_API_KEY"):
    raise SystemExit("Set ORS_API_KEY in your environment or .env before running this test.")

# Optional: bump logging from the client (INFO/DEBUG/WARNING/ERROR)
os.environ.setdefault("ORS_LOG_LEVEL", "INFO")

cfg = ORSConfig(default_country="BR")
ors = ORSClient(cfg)

# ──────────────────────────────────────────────────────────────────────────────
# Test cases
# ──────────────────────────────────────────────────────────────────────────────

# 1) Free-text address
run_case(
    "1) Free-text address",
    "Av. Paulista, 1578, São Paulo, SP",
    "Porto de Santos, Santos, SP",
)

# 2) CEP-only (postal code)
run_case(
    "2) CEP-only (postal code)",
    "01310-200",
    "11010-091",
)

# 3) City names
run_case(
    "3) City names",
    "São Paulo, SP",
    "Rio de Janeiro, RJ",
)

# 4) Coordinates (tuple) → to CEP
run_case(
    "4) Coordinates (tuple) → CEP",
    (-23.55052, -46.63331),
    "11010913",
)

# 5) Structured dict → explicit lat/lon
origin_struct = {
      "street": "Avenida Paulista"
    , "housenumber": "1578"
    , "locality": "São Paulo"
    , "region": "SP"
    , "postalcode": "01310-200"
    , "country": "BR"
}
run_case(
    "5) Structured dict",
    origin_struct,
    {"lat": -23.9608, "lon": -46.3336, "label": "Santos Ponta da Praia"},
)


[10:07:23] INFO cabosupernet.routes_provider | GEOCODE text='Av. Paulista, 1578, São Paulo, SP' country=BR size=1



1) Free-text address
--------------------------------------------------------------------------------
origin: Av. Paulista, 1578, São Paulo, SP
destination: Porto de Santos, Santos, SP


[10:07:30] INFO cabosupernet.routes_provider | RESOLVE (text) -> {'lat': -23.555036, 'lon': -46.663479, 'label': 'Avenida Paulista, São Paulo, Brazil'}
[10:07:30] INFO cabosupernet.routes_provider | GEOCODE text='Porto de Santos, Santos, SP' country=BR size=1
[10:07:35] INFO cabosupernet.routes_provider | RESOLVE (text) -> {'lat': -23.932511, 'lon': -46.324739, 'label': 'Alfandega do Porto de Santos, Santos, SP, Brazil'}
[10:07:35] INFO cabosupernet.routes_provider | ROUTE try1 driving-car coords=[[-46.663479, -23.555036], [-46.324739, -23.932511]]
[10:07:40] INFO cabosupernet.routes_provider | ROUTE OK km=77.39 hrs=1.23
[10:07:40] INFO cabosupernet.routes_provider | GEOCODE structured params={"country": "BR", "postalcode": "01310200", "size": 1}



✓ OK in 16849 ms — result:

{
  "destination": {
    "label": "Alfandega do Porto de Santos, Santos, SP, Brazil",
    "lat": -23.932511,
    "lon": -46.324739
  },
  "distance_m": 77393.1,
  "duration_s": 4421.9,
  "origin": {
    "label": "Avenida Paulista, São Paulo, Brazil",
    "lat": -23.555036,
    "lon": -46.663479
  },
  "segments": [
    {
      "distance": 77393.1,
      "duration": 4421.9,
      "steps": [
        {
          "distance": 135.9,
          "duration": 25.3,
          "instruction": "Head northwest on Avenida Paulista",
          "name": "Avenida Paulista",
          "type": 11,
          "way_points": [
            0,
            9
          ]
        },
        {
          "distance": 43.8,
          "duration": 12.9,
          "instruction": "Turn left onto Praça Marechal Cordeiro de Farias",
          "name": "Praça Marechal Cordeiro de Farias",
          "type": 0,
          "way_points": [
            9,
            13
          ]
        },
        {
  

[10:07:41] INFO cabosupernet.routes_provider | ViaCEP GET https://viacep.com.br/ws/01310200/json/
[10:07:42] INFO cabosupernet.routes_provider | RESOLVE (CEP ViaCEP->text) query='Avenida Paulista, Bela Vista, São Paulo, SP'
[10:07:42] INFO cabosupernet.routes_provider | GEOCODE text='Avenida Paulista, Bela Vista, São Paulo, SP' country=BR size=1
[10:07:43] INFO cabosupernet.routes_provider | RESOLVE (CEP via ViaCEP) -> {'lat': -23.563498, 'lon': -46.653985, 'label': 'Ciclovia da Avenida Paulista, São Paulo, Brazil'}
[10:07:43] INFO cabosupernet.routes_provider | GEOCODE structured params={"country": "BR", "postalcode": "11010091", "size": 1}
[10:07:48] INFO cabosupernet.routes_provider | ViaCEP GET https://viacep.com.br/ws/11010091/json/
[10:07:48] INFO cabosupernet.routes_provider | RESOLVE (CEP ViaCEP->text) query='Rua Frei Gaspar, Centro, Santos, SP'
[10:07:48] INFO cabosupernet.routes_provider | GEOCODE text='Rua Frei Gaspar, Centro, Santos, SP' country=BR size=1
[10:07:48] INFO ca

...


✓ OK in 1542 ms — result:

{
  "destination": {
    "label": "Rio de Janeiro, Brazil",
    "lat": -22.935024,
    "lon": -43.518246
  },
  "distance_m": 405461.5,
  "duration_s": 19172.8,
  "origin": {
    "label": "São Paulo, Brazil",
    "lat": -23.570533,
    "lon": -46.663713
  },
  "segments": [
    {
      "distance": 405461.5,
      "duration": 19172.8,
      "steps": [
        {
          "distance": 82.2,
          "duration": 19.7,
          "instruction": "Head northeast on Rua Paraguai",
          "name": "Rua Paraguai",
          "type": 11,
          "way_points": [
            0,
            1
          ]
        },
        {
          "distance": 74.6,
          "duration": 17.9,
          "instruction": "Turn right onto Rua Presidente Prudente",
          "name": "Rua Presidente Prudente",
          "type": 1,
          "way_points": [
            1,
            2
          ]
        },
        {
          "distance": 328.2,
          "duration": 32.1,
          "inst

[10:07:51] INFO cabosupernet.routes_provider | ViaCEP GET https://viacep.com.br/ws/11010913/json/
[10:07:51] INFO cabosupernet.routes_provider | RESOLVE (CEP ViaCEP->text) query='Rua Martin Affonso, Centro, Santos, SP'
[10:07:51] INFO cabosupernet.routes_provider | GEOCODE text='Rua Martin Affonso, Centro, Santos, SP' country=BR size=1
[10:07:51] INFO cabosupernet.routes_provider | RESOLVE (CEP via ViaCEP) -> {'lat': -23.935639, 'lon': -46.325439, 'label': 'Centro, Santos, SP, Brazil'}
[10:07:51] INFO cabosupernet.routes_provider | ROUTE try1 driving-car coords=[[-46.63331, -23.55052], [-46.325439, -23.935639]]
[10:07:52] INFO cabosupernet.routes_provider | ROUTE OK km=76.74 hrs=1.22
[10:07:52] INFO cabosupernet.routes_provider | GEOCODE structured params={"address": "1578 Avenida Paulista", "country": "BR", "locality": "São Paulo", "postalcode": "01310-200", "region": "SP", "size": 1}



✓ OK in 1339 ms — result:

{
  "destination": {
    "label": "Centro, Santos, SP, Brazil",
    "lat": -23.935639,
    "lon": -46.325439
  },
  "distance_m": 76737.4,
  "duration_s": 4385.9,
  "origin": {
    "label": "-23.550520,-46.633310",
    "lat": -23.55052,
    "lon": -46.63331
  },
  "segments": [
    {
      "distance": 76737.4,
      "duration": 4385.9,
      "steps": [
        {
          "distance": 145.7,
          "duration": 35.0,
          "instruction": "Head south on Praça da Sé",
          "name": "Praça da Sé",
          "type": 11,
          "way_points": [
            0,
            4
          ]
        },
        {
          "distance": 164.0,
          "duration": 22.0,
          "instruction": "Turn left onto Praça Doutor João Mendes",
          "name": "Praça Doutor João Mendes",
          "type": 0,
          "way_points": [
            4,
            8
          ]
        },
        {
          "distance": 579.3,
          "duration": 66.4,
          "instr

[10:07:56] INFO cabosupernet.routes_provider | RESOLVE (structured dict) -> {'lat': -23.555036, 'lon': -46.663479, 'label': 'Avenida Paulista, São Paulo, Brazil'}
[10:07:56] INFO cabosupernet.routes_provider | RESOLVE (dict lat/lon) -> {'lat': -23.9608, 'lon': -46.3336, 'label': 'Santos Ponta da Praia'}
[10:07:56] INFO cabosupernet.routes_provider | ROUTE try1 driving-car coords=[[-46.663479, -23.555036], [-46.3336, -23.9608]]
[10:07:56] INFO cabosupernet.routes_provider | ROUTE OK km=78.7 hrs=1.31



✓ OK in 4851 ms — result:

{
  "destination": {
    "label": "Santos Ponta da Praia",
    "lat": -23.9608,
    "lon": -46.3336
  },
  "distance_m": 78704.9,
  "duration_s": 4719.6,
  "origin": {
    "label": "Avenida Paulista, São Paulo, Brazil",
    "lat": -23.555036,
    "lon": -46.663479
  },
  "segments": [
    {
      "distance": 78704.9,
      "duration": 4719.6,
      "steps": [
        {
          "distance": 135.9,
          "duration": 25.3,
          "instruction": "Head northwest on Avenida Paulista",
          "name": "Avenida Paulista",
          "type": 11,
          "way_points": [
            0,
            9
          ]
        },
        {
          "distance": 43.8,
          "duration": 12.9,
          "instruction": "Turn left onto Praça Marechal Cordeiro de Farias",
          "name": "Praça Marechal Cordeiro de Farias",
          "type": 0,
          "way_points": [
            9,
            13
          ]
        },
        {
          "distance": 1033.5,
    